In [68]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import glob 
import cv2
import pickle 

In [69]:
from keras.models import Sequential
from keras.layers import Conv2D

In [70]:
import os 

In [71]:
print(os.listdir())

['.ipynb_checkpoints', 'annotated', 'day 13', 'day 15', 'day 16', 'day 19', 'Image Segmentation using Convolution filter and Random Forest.ipynb', 'images', 'RF_model.sav', 'test 001.jpg', 'test.jpg', 'train', 'Untitled.ipynb', 'venv']


In [72]:
SIZE = 512

In [73]:
train_images = []

In [74]:
for directory_path in glob.glob("images/train"):
    for img_path in glob.glob(os.path.join(directory_path,"*.tif")):
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        train_images.append(img)

In [75]:
train_images = np.array(train_images)

In [76]:
train_images

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ...,

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0,

In [77]:
train_masks = []

In [78]:
for directory_path in glob.glob("images/mask"):
    for mask_path in glob.glob(os.path.join(directory_path,"*.tif")):
        mask = cv2.imread(mask_path,0)
        mask = cv2.resize(mask,(SIZE,SIZE))
        train_masks.append(mask)
        

In [79]:
train_masks = np.array(train_masks)

In [80]:
train_masks

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [81]:
X_train  = train_images
y_train  = train_masks
y_train = np.expand_dims(y_train,axis=3)

In [82]:
activation = 'sigmoid'
feature_extractor = Sequential()
feature_extractor.add(Conv2D(32,3,activation=activation,padding='same',input_shape=(SIZE,SIZE,3)))
feature_extractor.add(Conv2D(32,3,activation=activation,padding='same',kernel_initializer='he_uniform'))

In [ ]:
X = feature_extractor.predict(X_train)

In [ ]:
X = X.reshape(-1,X.shape[3])

In [ ]:
Y = y_train.reshape(-1)

In [ ]:
dataset = pd.DataFrame(X)

In [ ]:
dataset['Label'] = Y

In [ ]:
print(dataset['Label'].unique())
print(dataset['Label'].value_counts())

In [ ]:
dataset = dataset[dataset['Label']!=0]

In [ ]:
X_for_RF = dataset.drop(labels = ['Label'],axis=1)

In [ ]:
Y_for_RF = dataset['Label']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
model.fit(X_for_RF,Y_for_RF)

In [ ]:
filename = 'RF_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model = pickle.load(open(filename,'rb'))

In [ ]:
test_img = cv2.imread('images/test/test 001.tif',cv2.IMREAD_COLOR)
test_img = cv2.resize(test_img,(SIZE,SIZE))
test_img = cv2.cvtColor(test_img,cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img,axis=0)

In [ ]:
X_test_feature = feature_extractor.predict(test_img)
X_test_feature = X_test_feature.reshape(-1,X_test_feature.shape[3])

In [ ]:
prediction = loaded_model.predict(X_test_feature)

In [ ]:
prediction_image = prediction.reshape(mask.shape)

In [ ]:
plt.imshow(prediction_image)
plt.savefig('test 001.jpg')